In [63]:
import pandas as pd
import numpy as np

from shapely.geometry import Point, shape, mapping
from shapely.ops import unary_union

import matplotlib.pyplot as plt
import seaborn as sns

#from geoband.API import *
import folium
import json
import geopandas as gpd
from geopandas import gpd

from folium import plugins
import re
from pandas import ExcelFile

In [64]:
data = pd.read_csv("data/finaldata_0618.csv",encoding = "cp949")

In [65]:
data.isnull().sum()

finedust      0
temp          0
humid         0
traffic       0
road          0
vent          0
pop           0
famperpop     0
foreign_rt    0
gu_in         0
gu_out        0
gugan_out     0
pop_dens      0
park          0
tree          0
river         0
bus           0
bike          0
trans         0
p10           0
p70           0
p20_60        0
dtype: int64

In [66]:
y = data["finedust"]
X = data.iloc[:,1:] 

In [67]:
from sklearn.model_selection import train_test_split
X_tr, X_tst, y_tr, y_tst = train_test_split(X,y,test_size = 0.2,random_state = 42)
print(X_tr.shape,y_tr.shape,X_tst.shape,y_tst.shape)

(844, 21) (844,) (212, 21) (212,)


In [68]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_tr)

X_tr = scaler.transform(X_tr)
X_tst = scaler.transform(X_tst)

In [69]:
X_tr = pd.DataFrame(X_tr)
X_tst = pd.DataFrame(X_tst)

In [70]:
data.columns[1:]

Index(['temp', 'humid', 'traffic', 'road', 'vent', 'pop', 'famperpop',
       'foreign_rt', 'gu_in', 'gu_out', 'gugan_out', 'pop_dens', 'park',
       'tree', 'river', 'bus', 'bike', 'trans', 'p10', 'p70', 'p20_60'],
      dtype='object')

In [71]:
X_tr.columns = data.columns[1:]
X_tst.columns = data.columns[1:]

### Ridge Lasso Regression

### Lasso

In [72]:
from sklearn import linear_model
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [73]:
lasso = LassoCV(cv = 5,random_state = 42)
lasso = lasso.fit(X_tr,y_tr)

In [74]:
y_tr_pred = lasso.predict(X_tr)
y_tst_pred = lasso.predict(X_tst)
lasso_tr_mse = mean_squared_error(list(y_tr),list(y_tr_pred))
lasso_tst_mse = mean_squared_error(list(y_tst),list(y_tst_pred))
print("lasso_tr_mse:",lasso_tr_mse)
print("lasso_tst_mse:",lasso_tst_mse)

lasso_tr_mse: 221.5815749953596
lasso_tst_mse: 177.52734111501348


In [75]:
pd.DataFrame({"col":X_tr.columns,"coef":lasso.coef_}).sort_values("coef")

,col,coef
0,temp,-3.127753
3,road,-1.154998
1,humid,-1.053923
19,p70,-0.248659
9,gu_out,-0.219492
18,p10,-0.000000
15,bus,0.000000
14,river,0.000000
13,tree,0.000000
12,park,-0.000000


In [76]:
r2_score(list(y_tr),y_tr_pred)

0.07014866995157831

In [77]:
r2_score(list(y_tst),y_tst_pred)

0.07673575434062974

In [78]:
pd.concat([pd.DataFrame(list(y_tst)),pd.DataFrame(y_tst_pred)],axis=1)

,0,0
0,51.217391,52.618080
1,78.835664,49.283833
2,37.194719,51.993001
3,58.404762,55.895974
4,53.244300,52.030963
...,...,...
207,52.650155,51.795019
208,64.758865,57.132017
209,54.105442,52.389838
210,34.346505,51.562233


### Ridge

In [79]:
ridge = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
ridge = ridge.fit(X_tr,y_tr)


In [80]:
y_tr_pred = ridge.predict(X_tr)
y_tst_pred = ridge.predict(X_tst)
ridge_tr_mse = mean_squared_error(y_tr,y_tr_pred)
ridge_tst_mse = mean_squared_error(y_tst,y_tst_pred)
print("ridge_tr_mse:",ridge_tr_mse)
print("ridge_tst_mse:",ridge_tst_mse)

ridge_tr_mse: 218.12721628089147
ridge_tst_mse: 175.00329417149513


In [81]:
pd.DataFrame({"col":X_tr.columns,"ridge_coef":ridge.coef_,"lasso_coef":lasso.coef_}).sort_values("lasso_coef")

,col,ridge_coef,lasso_coef
0,temp,-4.016800,-3.127753
3,road,-2.462194,-1.154998
1,humid,-1.705097,-1.053923
19,p70,-1.324789,-0.248659
9,gu_out,-1.703296,-0.219492
18,p10,0.063939,-0.000000
15,bus,0.168959,0.000000
14,river,0.316059,0.000000
13,tree,0.169794,0.000000
12,park,-0.280978,-0.000000


In [82]:
r2_score(list(y_tst),y_tst_pred)

0.08986253403934985

In [83]:
r2_score(list(y_tr),y_tr_pred)

0.0846446407703606

In [84]:
pd.concat([pd.DataFrame(list(y_tst)),pd.DataFrame(y_tst_pred)],axis=1)

,0,0
0,51.217391,51.825678
1,78.835664,49.038303
2,37.194719,53.402027
3,58.404762,55.338322
4,53.244300,52.522596
...,...,...
207,52.650155,51.791794
208,64.758865,57.474888
209,54.105442,51.068970
210,34.346505,48.436750


In [85]:
data = pd.read_csv("data/finaldata_0618.csv",encoding = "cp949")

In [86]:
y = data["finedust"]
X = data.iloc[:,1:] 